**Process pedestrian JSON file**

In [1]:
%matplotlib inline  
import os
import numpy as np
from PIL import Image
import cv2
from matplotlib.pyplot import figure, imshow, savefig
import matplotlib.pyplot as plt
import collections
import os
import json

In [4]:
fold_dict_filename = 'pedestrian_dataset_folds/fold_dict.json'
fold_dict = json.load(open(fold_dict_filename, 'r'))
num_frames = 30
frames_to_process = set()
for json_filename in fold_dict:
    json_path = os.path.join(fold_dict[json_filename], json_filename)
    ped_json = json.load(open(json_path, 'r'))
    video_name = ped_json['video']
    first_frame = ped_json['frame_data'][0]
    start = first_frame['frame_index']
    for idx in range(start, start + num_frames):
        frames_to_process.add(video_name + '-' + str(idx))

In [11]:
frames_to_process

{'video_0266-211',
 'video_0090-66',
 'video_0109-118',
 'video_0284-22',
 'video_0109-141',
 'video_0215-159',
 'video_0313-419',
 'video_0124-319',
 'video_0010-45',
 'video_0334-74',
 'video_0309-8',
 'video_0181-32',
 'video_0125-46',
 'video_0004-76',
 'video_0215-24',
 'video_0044-43',
 'video_0199-171',
 'video_0221-209',
 'video_0326-85',
 'video_0026-106',
 'video_0266-79',
 'video_0342-99',
 'video_0223-327',
 'video_0210-53',
 'video_0135-63',
 'video_0223-116',
 'video_0272-135',
 'video_0062-66',
 'video_0237-67',
 'video_0322-73',
 'video_0264-31',
 'video_0010-49',
 'video_0232-1',
 'video_0026-71',
 'video_0264-42',
 'video_0301-63',
 'video_0186-18',
 'video_0234-82',
 'video_0103-62',
 'video_0135-427',
 'video_0317-79',
 'video_0231-87',
 'video_0109-114',
 'video_0290-49',
 'video_0340-102',
 'video_0145-52',
 'video_0344-18',
 'video_0006-55',
 'video_0297-116',
 'video_0105-40',
 'video_0293-170',
 'video_0001-5',
 'video_0210-43',
 'video_0334-41',
 'video_0132-9

In [ ]:
frames_dict = collections.defaultdict(list)
for frame in frames_to_process:
    video, idx = frame.split('-')
    frames_dict[video].append

In [10]:
def getVideoFrames(videoName: str, frames: list):
    video = cv2.VideoCapture(videoName)
    i = 1
    while True:
        ret, img = cam.read()
        if not ret:
            return
        if i in frames:
            yield img
        i += 1

In [ ]:
for f